In [1]:
from serka.pipelines import PipelineBuilder

In [2]:
pb: PipelineBuilder = PipelineBuilder(neo4j_user="neo4j", neo4j_password="password")

In [3]:
from haystack_integrations.components.generators.ollama.generator import OllamaGenerator
from haystack import Pipeline
from haystack.components.builders import PromptBuilder
from haystack.components.routers import ConditionalRouter
from haystack.components.builders.answer_builder import AnswerBuilder
from haystack.components.joiners import StringJoiner

In [4]:
QUERY_TYPE_PROMPT = """
You are a helpful assistant.
You are part of a RAG pipeline that processes queries and answers questions about environmental science data held in the EIDC (Environmental Information Data Centre).
Your task is to classify a query from a user based on it's content so that it can be processed by the appropriate pipeline.
You must consider whether or not the query is related to environmental science or if the query could feasibly be answered by infroamtion in the EIDC.
You should return a single word that describes the type of query from the following list:
- GENERAL
- DESCRIPTIVE
- CODE
- METADATA
- UNRELATED

You should not return any other text, just the single word.

Conditions when to apply certain labels:
- GENERAL: The query is a general question about environmental science such as biodiversity, climate change, or conservation.
- DESCRIPTIVE: The query is asking for a description of a specific dataset or information about a particular environmental topic.
- CODE: The query is asking you to generate code or programming help related to environmental data processing or analysis.
- METADATA: The query is asking for metadata information about datasets, such as their source, date, or who authored them.
- UNRELATED: The query is not related to environmental science and is unlikely to be answerable from the information in the EIDC.

The user's query is:
{{query}}
"""

In [5]:
p: Pipeline = Pipeline()
p.add_component("query_type_prompt_builder", PromptBuilder(QUERY_TYPE_PROMPT))
p.add_component(
	"llm",
	OllamaGenerator(
		model="llama3.1",
		generation_kwargs={"num_ctx": 16384, "temperature": 0.0},
		url="http://localhost:11434",
	),
)
p.add_component(
	"router",
	ConditionalRouter(
		routes=[
			{
				"condition": "{{'UNRELATED' in replies[0]}}",
				"output": 'I\'m afraid your query: "{{query}}" does not appear to be related to environmental science or the EIDC. Please try asking a different question.',
				"output_name": "unrelated_query",
				"output_type": str,
			},
			{
				"condition": "{{'CODE' in replies[0]}}",
				"output": "Unfortunately I can't generate code. I can try to help you find environmental science data in the EIDC.",
				"output_name": "code_query",
				"output_type": str,
			},
			{
				"condition": "{{'UNRELATED' not in replies[0]}}",
				"output": "{{query}}",
				"output_name": "handle_query",
				"output_type": str,
			},
		]
	),
)
p.add_component("joiner", StringJoiner())
p.add_component("answer_builder", AnswerBuilder())


p.connect("query_type_prompt_builder", "llm")
p.connect("llm.replies", "router.replies")
p.connect("router.unrelated_query", "joiner")
p.connect("router.code_query", "joiner")

p.connect("joiner", "answer_builder.replies")

🚅 Components
  - query_type_prompt_builder: PromptBuilder
  - llm: OllamaGenerator
  - router: ConditionalRouter
  - joiner: StringJoiner
  - answer_builder: AnswerBuilder
🛤️ Connections
  - query_type_prompt_builder.prompt -> llm.prompt (str)
  - llm.replies -> router.replies (List[str])
  - router.unrelated_query -> joiner.strings (str)
  - router.code_query -> joiner.strings (str)
  - joiner.strings -> answer_builder.replies (List[str])

In [1]:
query = "What chemicals make up smog?"

In [ ]:
p.run(
	data={
		"query_type_prompt_builder": {"query": query},
		"router": {"query": query},
		"answer_builder": {"query": query},
		"invalid_answer_builder": {"query": query},
	}
)

NameError: name 'p' is not defined

In [1]:
from serka.pipelines import PipelineBuilder
from haystack import Pipeline

In [6]:
pb: PipelineBuilder = PipelineBuilder(neo4j_user="neo4j", neo4j_password="password")
rag_pipe: Pipeline = pb.rag_pipeline()
rag_pipe.draw(path="rag_pip.png")

2025-05-30 15:19:50,241 - haystack.components.builders.prompt_builder - WARNING - PromptBuilder has 1 prompt variables, but `required_variables` is not set. By default, all prompt variables are treated as optional, which may lead to unintended behavior in multi-branch pipelines. To avoid unexpected execution, ensure that variables intended to be required are explicitly set in `required_variables`.
2025-05-30 15:19:50,261 - haystack.components.builders.prompt_builder - WARNING - PromptBuilder has 2 prompt variables, but `required_variables` is not set. By default, all prompt variables are treated as optional, which may lead to unintended behavior in multi-branch pipelines. To avoid unexpected execution, ensure that variables intended to be required are explicitly set in `required_variables`.


In [3]:
query = "Describe the land cover map dataset"

In [4]:
rag_pipe.run(
	data={
		"query_type_prompt_builder": {"query": query},
		"router": {"query": query},
		"invalid_answer_builder": {"query": query},
	}
)

2025-05-30 15:09:43,884 - haystack.core.pipeline.pipeline - INFO - Running component query_type_prompt_builder
2025-05-30 15:09:43,885 - haystack.core.pipeline.pipeline - INFO - Running component llm_type_classifier
2025-05-30 15:09:43,995 - httpx - INFO - HTTP Request: POST http://localhost:11434/api/generate "HTTP/1.1 200 OK"
2025-05-30 15:09:43,996 - haystack.core.pipeline.pipeline - INFO - Running component router
2025-05-30 15:09:43,998 - haystack.core.pipeline.pipeline - INFO - Running component embedder
2025-05-30 15:09:44,988 - httpx - INFO - HTTP Request: POST http://localhost:11434/api/embeddings "HTTP/1.1 200 OK"
2025-05-30 15:09:44,990 - haystack.core.pipeline.pipeline - INFO - Running component reader
2025-05-30 15:09:46,387 - neo4j.notifications - WARNING - Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future ve

{'llm_type_classifier': {'meta': [{'model': 'llama3.1',
    'created_at': '2025-05-30T14:09:43.993622554Z',
    'done': True,
    'done_reason': 'stop',
    'total_duration': 106360090,
    'load_duration': 19668745,
    'prompt_eval_count': 295,
    'prompt_eval_duration': 23442427,
    'eval_count': 4,
    'eval_duration': 62509708,
    'context': [128006,
     882,
     128007,
     1432,
     2675,
     527,
     264,
     11190,
     18328,
     627,
     2675,
     527,
     961,
     315,
     264,
     432,
     1929,
     15660,
     430,
     11618,
     20126,
     323,
     11503,
     4860,
     922,
     12434,
     8198,
     828,
     5762,
     304,
     279,
     469,
     926,
     34,
     320,
     83166,
     8245,
     2956,
     14821,
     4390,
     7927,
     3465,
     374,
     311,
     49229,
     264,
     3319,
     505,
     264,
     1217,
     3196,
     389,
     433,
     596,
     2262,
     779,
     430,
     433,
     649,
     387,
     15590,